In [1]:
import os
import pandas as pd
import subprocess
import re
import math

# Ensure we are in the project root
if os.getcwd().endswith("src"):
    os.chdir("..")
    
print(f"Current working directory: {os.getcwd()}")

Current working directory: /Users/dkandboz/Documents/college/deep-learning-gnn-project


## Config

In [2]:
from dataclasses import dataclass


@dataclass
class Config:
    RUN_4_1_GNN: bool = False
    RUN_4_2_MEAN_TEACHER: bool = True
    RUN_4_3_NCPS: bool = False
    RUN_4_5_GRAPHMIX: bool = False
    RUN_6_FINAL_TEST: bool = False
    SHOULD_TRAIN: bool = False
    
CONFIG = Config()

## Utils

In [3]:
def run_command(command):
    """Runs a shell command and returns the output."""
    print(f"Running: {command}")
    # Capture both stdout and stderr
    process = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    if process.returncode != 0:
        print(f"Error running command: {command}")
        print("STDERR:", process.stderr)
    
    return process.stdout

def parse_mse(output):
    """Parses the FINAL_TEST_MSE from the output of src/test.py"""
    # Look for the specific marker we added to src/test.py
    match = re.search(r"FINAL_TEST_MSE: ([\d\.]+)", output)
    if match:
        return float(match.group(1))
    return None

## 4.1 GNN Ablation study

In [4]:
def gnn_ablation_study():   
    if not CONFIG.RUN_4_1_GNN:
        print("Skipping GNN Ablation Study as per configuration.")
        return []
    experiments_list = [
        {
            "id": "1",
            "label": "Baseline GCN (StepLR, No Reg.)",
            "config": "ablation_gnn/01_gcn_baseline",
            "phase": "Phase 1: Regularization & Optimization (GCN Backbone)",
        },
        {
            "id": "2",
            "label": "+ Residual Connections",
            "config": "ablation_gnn/02_gcn_residuals",
            "phase": "Phase 1: Regularization & Optimization (GCN Backbone)",
        },
        {
            "id": "3a",
            "label": "+ Batch Normalization",
            "config": "ablation_gnn/03a_gcn_batchnorm",
            "phase": "Phase 1: Regularization & Optimization (GCN Backbone)",
        },
        {
            "id": "3b",
            "label": "+ Layer Normalization",
            "config": "ablation_gnn/03b_gcn_layernorm",
            "phase": "Phase 1: Regularization & Optimization (GCN Backbone)",
        },
        {
            "id": "3c",
            "label": "+ Graph Normalization",
            "config": "ablation_gnn/03c_gcn_graphnorm",
            "phase": "Phase 1: Regularization & Optimization (GCN Backbone)",
        },
        {
            "id": "4",
            "label": "+ Dropout (p=0.1)",
            "config": "ablation_gnn/04_gcn_dropout",
            "phase": "Phase 1: Regularization & Optimization (GCN Backbone)",
        },
        {
            "id": "5",
            "label": "+ Cosine Annealing Scheduler",
            "config": "ablation_gnn/05_gcn_cosine",
            "phase": "Phase 1: Regularization & Optimization (GCN Backbone)",
        },
        {
            "id": "6a",
            "label": "GCN (Reference)",
            "config": "ablation_gnn/06a_gcn",
            "phase": "Phase 2: Model Selection (Fixed Hyperparams)",
        },
        {
            "id": "6b",
            "label": "GraphSAGE",
            "config": "ablation_gnn/06b_sage",
            "phase": "Phase 2: Model Selection (Fixed Hyperparams)",
        },
        {
            "id": "6c",
            "label": "GAT",
            "config": "ablation_gnn/06c_gat",
            "phase": "Phase 2: Model Selection (Fixed Hyperparams)",
        },
        {
            "id": "6d",
            "label": "GIN",
            "config": "ablation_gnn/06d_gin",
            "phase": "Phase 2: Model Selection (Fixed Hyperparams)",
        },
        {
            "id": "7",
            "label": "GIN + Jumping Knowledge (Concat)",
            "config": "ablation_gnn/07_gin_jk",
            "phase": "Phase 3: Scaling & Refinement (GIN Backbone)",
        },
        {
            "id": "8",
            "label": "Deep GIN (12 Layers, 512 Hidden)",
            "config": "ablation_gnn/08_deep_gin",
            "phase": "Phase 3: Scaling & Refinement (GIN Backbone)",
        },
    ]

    results = []

    for exp in experiments_list:
        exp_id = exp["id"]
        label = exp["label"]
        cfg = exp["config"]
        phase = exp["phase"]

        print(f"\n=== Processing {exp_id} - {label} ===")

        # 1. Training (optional)
        if CONFIG.SHOULD_TRAIN:
            train_cmd = f"python src/run.py +experiments={cfg} trainer.train.total_epochs=1 save_model=false logger.disable=true"
            run_command(train_cmd)

        # 2. Testing
        # By default, we expect model files in models/<config>.pt
        # for example: models/ablation_gnn/01_gcn_baseline.pt
        model_path = f"models/{cfg}.pt"

        if not os.path.exists(model_path) and not CONFIG.SHOULD_TRAIN:
            print(f"Warning: Model file not found at {model_path}. Skipping evaluation.")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": None}
            )
            continue

        test_cmd = f"python src/test.py +experiments={cfg} model_path={model_path}"
        output = run_command(test_cmd)
        mse = parse_mse(output)

        if mse is not None:
            print(f"Parsed Test MSE: {mse}")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": mse}
            )
        else:
            print("Could not parse MSE from output.")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": None}
            )
    return results

In [5]:
results = gnn_ablation_study()

Skipping GNN Ablation Study as per configuration.


In [6]:
def generate_latex_table(results):
    if not results:
        print("No results to generate LaTeX table.")
        return
    df_results = pd.DataFrame(results)

    # Clean up Configuration column
    if not df_results.empty:
        df_results["Configuration Display"] = df_results["Configuration"].astype(str)

    # --- FIXED: Robust LaTeX Escape Function ---
    def latex_escape(s):
        """
        Escapes special characters for LaTeX using a single-pass regex 
        to prevent double-escaping (e.g. replacing '\' then '{').
        """
        if s is None:
            return ""
        s = str(s)
        
        # Map of characters to their LaTeX escape sequences
        # Note: Order doesn't matter in the regex approach
        chars = {
            '&': r'\&',
            '%': r'\%',
            '$': r'\$',
            '#': r'\#',
            '_': r'\_',
            '{': r'\{',
            '}': r'\}',
            '~': r'\textasciitilde{}',
            '^': r'\textasciicircum{}',
            '\\': r'\textbackslash{}',
        }
        
        # Create a regex that matches any of the keys
        pattern = re.compile('|'.join(re.escape(k) for k in chars.keys()))
        
        # Substitute using the dictionary
        return pattern.sub(lambda m: chars[m.group(0)], s)

    # Group by phase
    phases = df_results["Phase"].dropna().unique().tolist() if not df_results.empty else []

    # Find best overall MSE for bolding
    valid_mses = df_results["Test MSE"][df_results["Test MSE"].notnull()].tolist()
    best_overall = min(valid_mses) if valid_mses else None

    lines = []
    lines.append(r"\begin{table}[h!]")
    lines.append(r"    \centering")
    lines.append(r"    \caption{Ablation study of supervised GNN architecture and regularization techniques on QM9 dataset.}")
    lines.append(r"    \label{tab:gnn_ablation}")
    # Resize box to fit column width
    lines.append(r"    \resizebox{\columnwidth}{!}{%")
    lines.append(r"    \begin{tabular}{llc}")
    lines.append(r"        \toprule")
    lines.append(r"        \textbf{ID} & \textbf{Configuration / Modification} & \textbf{Test MSE} \\")
    lines.append(r"        \midrule")

    for i, phase in enumerate(phases):
        phase_escaped = latex_escape(phase)
        
        # --- FIXED: Correct string syntax for multicolumn ---
        # Using an f-string with double braces {{ }} for LaTeX literals
        # and 4 backslashes \\\\ to produce a double backslash \\ in the output
        lines.append(f"        \\multicolumn{{3}}{{l}}{{\\textit{{{phase_escaped}}}}} \\\\")
        
        phase_rows = df_results[df_results["Phase"] == phase]
        
        for _, row in phase_rows.iterrows():
            id_ = row["ID"]
            cfg = row["Configuration Display"]
            cfg_escaped = latex_escape(cfg)
            mse = row["Test MSE"]
            
            if mse is None or (isinstance(mse, float) and math.isnan(mse)):
                mse_str = "--"
            else:
                mse_fmt = f"{mse:.4f}"
                # Bold if best
                if best_overall is not None and abs(mse - best_overall) < 1e-12:
                    mse_str = r"\textbf{" + mse_fmt + "}"
                else:
                    mse_str = mse_fmt
            
            # --- FIXED: Row Ending ---
            # 4 backslashes \\\\ in python string -> 2 backslashes \\ in output file (LaTeX newline)
            lines.append(f"        {id_} & {cfg_escaped} & {mse_str} \\\\")
        
        # Add midrule only if it's not the last phase (standard booktabs style)
        if i < len(phases) - 1:
            lines.append(r"        \midrule")

    lines.append(r"        \bottomrule")
    lines.append(r"    \end{tabular}%")
    lines.append(r"    }")
    lines.append(r"\end{table}")

    latex_table = "\n".join(lines)
    print(latex_table)
    
generate_latex_table(results)

No results to generate LaTeX table.


## 4.2 Mean Teacher

In [7]:
def mean_teacher():   
    if not CONFIG.RUN_4_2_MEAN_TEACHER:
        print("Skipping Mean Teacher as per configuration.")
        return []
    experiments_list = [
        {
            "id": "1",
            "label": "Mean Teacher Lambda=0",
            "config": "mean_teacher/mean_teacher_0",
            "phase": "Mean Teacher",
        },
        {
            "id": "2",
            "label": "Mean Teacher Lambda=10e-4",
            "config": "mean_teacher/mean_teacher_10e-4",
            "phase": "Mean Teacher",
        },
        {
            "id": "3",
            "label": "Mean Teacher Lambda=10e-3 (10% Labelled Data)",
            "config": "mean_teacher/mean_teacher_10e-3",
            "phase": "Mean Teacher",
        },
        {
            "id": "4",
            "label": "Mean Teacher Lambda=10e-3 1% Labelled Data",
            "config": "mean_teacher/mean_teacher_10e-3_1p",
            "phase": "Mean Teacher",
        },
        {
            "id": "5",
            "label": "Mean Teacher Lambda=10e-3 5% Labelled Data",
            "config": "mean_teacher/mean_teacher_10e-3_5p",
            "phase": "Mean Teacher",
        },
        {
            "id": "6",
            "label": "Mean Teacher Lambda=10e-3 20% Labelled Data",
            "config": "mean_teacher/mean_teacher_10e-3_20p",
            "phase": "Mean Teacher",
        },
    ]

    results = []

    for exp in experiments_list:
        exp_id = exp["id"]
        label = exp["label"]
        cfg = exp["config"]
        phase = exp["phase"]

        print(f"\n=== Processing {exp_id} - {label} ===")

        # 1. Training (optional)
        if CONFIG.SHOULD_TRAIN:
            train_cmd = f"python src/run.py +experiments={cfg} trainer.train.total_epochs=1 save_model=false logger.disable=true"
            run_command(train_cmd)

        # 2. Testing
        # By default, we expect model files in models/<config>.pt
        # for example: models/ablation_gnn/01_gcn_baseline.pt
        model_path = f"models/{cfg}.pt"

        if not os.path.exists(model_path) and not CONFIG.SHOULD_TRAIN:
            print(f"Warning: Model file not found at {model_path}. Skipping evaluation.")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": None}
            )
            continue

        test_cmd = f"python src/test.py +experiments={cfg} model_path={model_path}"
        output = run_command(test_cmd)
        mse = parse_mse(output)

        if mse is not None:
            print(f"Parsed Test MSE: {mse}")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": mse}
            )
        else:
            print("Could not parse MSE from output.")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": None}
            )
    return results

In [8]:
results = mean_teacher()


=== Processing 1 - Mean Teacher Lambda=0 ===
Running: python src/test.py +experiments=mean_teacher/mean_teacher_0 model_path=models/mean_teacher/mean_teacher_0.pt
Parsed Test MSE: 7.678863797869001

=== Processing 2 - Mean Teacher Lambda=10e-4 ===
Running: python src/test.py +experiments=mean_teacher/mean_teacher_10e-4 model_path=models/mean_teacher/mean_teacher_10e-4.pt
Parsed Test MSE: 8.552429471697126

=== Processing 3 - Mean Teacher Lambda=10e-3 (10% Labelled Data) ===
Running: python src/test.py +experiments=mean_teacher/mean_teacher_10e-3 model_path=models/mean_teacher/mean_teacher_10e-3.pt
Parsed Test MSE: 8.350629125322614

=== Processing 4 - Mean Teacher Lambda=10e-3 1% Labelled Data ===
Running: python src/test.py +experiments=mean_teacher/mean_teacher_10e-3_1p model_path=models/mean_teacher/mean_teacher_10e-3_1p.pt
Parsed Test MSE: 5.04703106198992

=== Processing 5 - Mean Teacher Lambda=10e-3 5% Labelled Data ===
Running: python src/test.py +experiments=mean_teacher/mean_

In [16]:
display(results)

[]

## 4.3 NCPS

In [ ]:
def ncps():   
    if not CONFIG.RUN_4_3_NCPS:
        print("Skipping NCPS as per configuration.")
        return []
    experiments_list = [
        {
            "id": "1",
            "label": "GCN CPS",
            "config": "cps/cps_gcn_1000",
            "phase": "",
        },
        {
            "id": "2",
            "label": "GINC CPS",
            "config": "cps/cps_gin_250",
            "phase": "",
        }
    ]

    results = []

    for exp in experiments_list:
        exp_id = exp["id"]
        label = exp["label"]
        cfg = exp["config"]
        phase = exp["phase"]

        print(f"\n=== Processing {exp_id} - {label} ===")

        # 1. Training (optional)
        if CONFIG.SHOULD_TRAIN:
            train_cmd = f"python src/run_cps.py +experiments={cfg} trainer.train.total_epochs=1 save_model=false logger.disable=true"
            run_command(train_cmd)

        # 2. Testing
        # By default, we expect model files in models/<config>.pt
        # for example: models/ablation_gnn/01_gcn_baseline.pt
        model_path = f"models/{cfg}.pt"

        if not os.path.exists(model_path) and not CONFIG.SHOULD_TRAIN:
            print(f"Warning: Model file not found at {model_path}. Skipping evaluation.")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": None}
            )
            continue

        test_cmd = f"python src/test.py +experiments={cfg} model_path={model_path}"
        output = run_command(test_cmd)
        mse = parse_mse(output)

        if mse is not None:
            print(f"Parsed Test MSE: {mse}")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": mse}
            )
        else:
            print("Could not parse MSE from output.")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": None}
            )
    return results
            

In [10]:
results = ncps()

Skipping NCPS as per configuration.


## 4.5 Graph Mix

In [11]:
def graph_mix():
    if not CONFIG.RUN_4_5_GRAPHMIX:
        print("Skipping GraphMix as per configuration.")
        return []
    
    group_1 = "GraphMix"
    group_2 = "Supervised"
    experiments_list = [
        {
            "id": "1",
            "label": "GraphMix 1% Labelled Data",
            "config": "graph_mixup/gin_best_mixup_1",
            "phase": group_1,
        },
        {
            "id": "2",
            "label": "GraphMix 5% Labelled Data",
            "config": "graph_mixup/gin_best_mixup_5",
            "phase": group_1,
        },
        {
            "id": "3",
            "label": "GraphMix 10% Labelled Data",
            "config": "graph_mixup/gin_best_mixup",
            "phase": group_1,
        },
        {
            "id": "4",
            "label": "GraphMix 20% Labelled Data",
            "config": "graph_mixup/gin_best_mixup_20",
            "phase": group_1,
        },
        {
            "id": "5",
            "label": "Supervised 1% Labelled Data",
            "config": "gin_best_1",
            "phase": group_2,
        },
        {
            "id": "6",
            "label": "Supervised 5% Labelled Data",
            "config": "gin_best_5",
            "phase": group_2,
        },
        {
            "id": "7",
            "label": "Supervised 10% Labelled Data",
            "config": "gin_best",
            "phase": group_2,
        },
        {
            "id": "8",
            "label": "Supervised 20% Labelled Data",
            "config": "gin_best_20",
            "phase": group_2,
        },
    ]

    results = []

    for exp in experiments_list:
        exp_id = exp["id"]
        label = exp["label"]
        cfg = exp["config"]
        phase = exp["phase"]

        print(f"\n=== Processing {exp_id} - {label} ===")

        # 1. Training (optional)
        if CONFIG.SHOULD_TRAIN:
            train_cmd = f"python src/run.py +experiments={cfg} trainer.train.total_epochs=1 save_model=false logger.disable=true"
            run_command(train_cmd)

        # 2. Testing
        # By default, we expect model files in models/<config>.pt
        # for example: models/ablation_gnn/01_gcn_baseline.pt
        model_path = f"models/{cfg}.pt"

        if not os.path.exists(model_path) and not CONFIG.SHOULD_TRAIN:
            print(
                f"Warning: Model file not found at {model_path}. Skipping evaluation."
            )
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": None}
            )
            continue

        test_cmd = f"python src/test.py +experiments={cfg} model_path={model_path}"
        output = run_command(test_cmd)
        mse = parse_mse(output)

        if mse is not None:
            print(f"Parsed Test MSE: {mse}")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": mse}
            )
        else:
            print("Could not parse MSE from output.")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": None}
            )
    return results

In [12]:
results = graph_mix()
display(results)

Skipping GraphMix as per configuration.


[]

In [13]:
# Build a comparative LaTeX table: Supervised vs GraphMix for different labelled-data proportions.
# Uses `results` from previous cells.

def _latex_escape(s):
    if s is None:
        return ""
    s = str(s)
    chars = {
        '&': r'\&',
        '%': r'\%',
        '$': r'\$',
        '#': r'\#',
        '_': r'\_',
        '{': r'\{',
        '}': r'\}',
        '~': r'\textasciitilde{}',
        '^': r'\textasciicircum{}',
        '\\': r'\textbackslash{}',
    }
    pattern = re.compile('|'.join(re.escape(k) for k in chars.keys()))
    return pattern.sub(lambda m: chars[m.group(0)], s)

if not results:
    print("No results available to build LaTeX table.")
else:
    # Identify entries by phase names used in graph_mix()
    GRAPHMIX_PHASE = "GraphMix"
    SUP_PHASE = "Supervised"

    # Percentages to compare (strings as in labels)
    percents = ["1%", "5%", "10%", "20%"]

    # Helper to find MSE for a given phase and percent
    def find_mse(phase, percent):
        for r in results:
            if r.get("Phase") == phase and percent in str(r.get("Configuration", "")):
                return r.get("Test MSE")
        return None

    lines = []
    lines.append(r"\begin{table}[h]")
    lines.append(r"  \centering")
    lines.append(r"  \caption{Comparison: GraphMix vs Supervised baseline across different labelled-data proportions}")
    lines.append(r"  \begin{tabular}{lcc}")
    lines.append(r"    \toprule")
    lines.append(r"    Labelled Data & GraphMix (MSE) & Supervised (MSE) \\")
    lines.append(r"    \midrule")

    for pct in percents:
        gmse = find_mse(GRAPHMIX_PHASE, pct)
        smse = find_mse(SUP_PHASE, pct)

        def fmt(m):
            if m is None or (isinstance(m, float) and math.isnan(m)):
                return "--"
            return f"{m:.4f}"

        # Determine which to bold (lower MSE is better)
        gm_val = None if gmse is None else float(gmse)
        sm_val = None if smse is None else float(smse)

        gm_str = fmt(gm_val)
        sm_str = fmt(sm_val)

        if gm_val is not None and sm_val is not None:
            if abs(gm_val - sm_val) < 1e-12:
                # tie -> bold both
                gm_str = r"\textbf{" + f"{gm_val:.4f}" + "}"
                sm_str = r"\textbf{" + f"{sm_val:.4f}" + "}"
            elif gm_val < sm_val:
                gm_str = r"\textbf{" + f"{gm_val:.4f}" + "}"
            else:
                sm_str = r"\textbf{" + f"{sm_val:.4f}" + "}"

        lines.append(f"    {_latex_escape(pct)} & {gm_str} & {sm_str} \\\\")

    lines.append(r"    \bottomrule")
    lines.append(r"  \end{tabular}")
    lines.append(r"\end{table}")

    latex_table = "\n".join(lines)
    print(latex_table)

No results available to build LaTeX table.


## 6 Conclusion

In [14]:
def run_best_test():   
    if not CONFIG.RUN_6_FINAL_TEST:
        print("Skipping GNN Ablation Study as per configuration.")
        return []
    experiments_list = [
        {
            "id": "1",
            "label": "Best Model",
            "config": "gin_best",
            "phase": "",
        },
    ]

    results = []

    for exp in experiments_list:
        exp_id = exp["id"]
        label = exp["label"]
        cfg = exp["config"]
        phase = exp["phase"]

        print(f"\n=== Processing {exp_id} - {label} ===")

        # 1. Training (optional)
        if CONFIG.SHOULD_TRAIN:
            train_cmd = f"python src/run.py +experiments={cfg} trainer.train.total_epochs=1 save_model=false logger.disable=true"
            run_command(train_cmd)

        # 2. Testing
        # By default, we expect model files in models/<config>.pt
        # for example: models/ablation_gnn/01_gcn_baseline.pt
        model_path = f"models/{cfg}.pt"

        if not os.path.exists(model_path) and not CONFIG.SHOULD_TRAIN:
            print(f"Warning: Model file not found at {model_path}. Skipping evaluation.")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": None}
            )
            continue

        test_cmd = f"python src/test.py +experiments={cfg} model_path={model_path} use_test=true"
        output = run_command(test_cmd)
        mse = parse_mse(output)

        if mse is not None:
            print(f"Parsed Test MSE: {mse}")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": mse}
            )
        else:
            print("Could not parse MSE from output.")
            results.append(
                {"ID": exp_id, "Configuration": label, "Phase": phase, "Test MSE": None}
            )
    return results

In [15]:
results = run_best_test()
display(results)

Skipping GNN Ablation Study as per configuration.


[]